In [3]:
import os
import typing
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib

from typing import Tuple

sns.set_theme()
plt.figure(figsize = (22,16))

<Figure size 2200x1600 with 0 Axes>

<Figure size 2200x1600 with 0 Axes>

In [4]:
# file management to set up data directory
current_folder = globals()['_dh'][0]
project_folder = "/".join(str(current_folder).split("/")[:-1])
data_directory = os.path.join(project_folder, "data")

In [5]:
os.listdir(data_directory)

['sbab_mortgage_pricing_11_04_23.csv',
 'ica_mortgage_pricing_20_04_23.csv',
 'sbab_mortgage_pricing_10_04_23.csv',
 '.gitkeep',
 'ica_mortgage_pricing_30_03_23.csv',
 'sbab_mortgage_pricing_13_04_23.csv',
 'sbab_mortgage_pricing_20_04_23.csv',
 'sbab_mortgage_pricing_06_04_23.csv']

In [6]:
# set up data import
filepath = os.path.join(data_directory, "sbab_mortgage_pricing_20_04_23.csv")
df = pd.read_csv(filepath, index_col=0)

In [7]:
# standardise and strip superflous columns
superflous_cols = ["provider", "effektivrantesats", "loptidtext"]
relevant_cols = [c for c in df.columns if c not in superflous_cols]

df = df[relevant_cols]
df.columns = [c.lower().strip() for c in df.columns]

clean_columns_names = {
    "rantesats": "interest_rate",
    "rantebindningstid": "period",
    "loan_amount": "loan_volume",
}

df = df.rename(columns=clean_columns_names)
df.head()

,loptidtext,interest_rate,period,effektivrantesats,loan_volume,asset_value
0,2023-07-24,4.17,3,0.0,50000,50000
1,2024-04-24,4.92,12,0.0,50000,50000
2,2025-05-19,4.91,24,0.0,50000,50000
3,2026-05-19,4.73,36,0.0,50000,50000
4,2027-05-19,4.65,48,0.0,50000,50000


In [8]:
# adding LTV ratios and dropping rows with LTVs > 1
df["ltv"] = df.loan_volume / df.asset_value
df = df.loc[df["ltv"] <= 1]

In [9]:
# subset into different maturity periods ltv > 0.5
mask = (df.period == 3) & (df.ltv >= 0.5)
three_month_df = df.loc[mask].drop(["period", "asset_value"], axis=1)

# adding discount (ambigious, but inferred through the max rate)
three_month_df["discount"] = three_month_df["interest_rate"] - three_month_df["interest_rate"].max()

In [10]:
# we should get different interest rates different tuples of LTV,Volume, i.e. 2-dimensional pricing.
# Should ~29 acc. to prior exploration
three_month_df.interest_rate.value_counts().shape 

(29,)

In [11]:
three_month_df[["loan_volume", "ltv", "discount"]].ltv

0         1.000000
8         0.500000
1608      1.000000
1616      0.666667
1624      0.500000
            ...   
316784    0.994975
316792    0.990000
318384    1.000000
318392    0.995000
319992    1.000000
Name: ltv, Length: 10200, dtype: float64

### Pricing surface generation

In [12]:
df = three_month_df[["loan_volume", "ltv", "discount"]].loc[three_month_df.ltv > 0.5]
df = df.sort_values(by="loan_volume", ascending=False)

### import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (20,12)

z_min, z_max = df.discount.min(), df.discount.max()
fig, ax = plt.subplots()

x = df.ltv.sort()
y = df.loan_volume.sort(a)
z = df.pivot(columns=["loan_volume", "ltv"]).fillna(0).values
c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z.min(), vmax=z.max())
ax.set_title('pcolormesh')

# set the limits of the plot to the limits of the data
ax.axis([df.ltv.min(), df.ltv.max(), df.loan_volume.max(), df.loan_volume.min()])
fig.colorbar(c, ax=ax)

plt.show()